MedOptix AI - A project on Health sector to predict patients who are likely to dropout of therapy

- Import All Necessary Libraries

In [179]:
# Import dependencies
import boto3
import os
import sqlalchemy
import psycopg2
import pandas as pd


In [195]:
import boto3
print(boto3.__version__)

1.39.3


- (A) - Upload CSV files to AWS S3 (using boto3)

In [189]:
# Setup
# Define variable (AKA parameters)
bucket_name = "flo-amdari-demo-etl"
folder_data = "../medoptix_data/raw"
target_folder = "/medoptix/raw/"

# AWS credentials assumed to be in ~/.aws/credentials or env vars
# Activate a boto3 client
s3 = boto3.client("s3")

# Upload all CSVs in folder with ACL: bucket-owner-full-control 
# Looped into data folder and uploaded it to s3

try:
    for filename in os.listdir(folder_data):                # looped into the data folder
        if filename.endswith(".csv"):                       # check for only files that ended with .csv
            filepath = os.path.join(folder_data, filename)  # define a path (eg medoptix_data/raw/patients.csv)
            s3.upload_file(
                Filename = filepath,                        # used the client method (upload_file) to upload all the csv files into our s3 bucket)
                Bucket = bucket_name,
                Key = target_folder + filename              # combination of the path(s3) plus the filename (medoptix/raw/patients.csv)
                #ExtraArgs={'ACL': 'bucket-owner-full-control'}  # <-- THIS IS KEY
            )
            #print(f"✅ Uploaded {file_name} to s3://{bucket_name}/{folder_prefix}{file_name} with full access")
            
            print(f"uploaded {filename} successfully")      # printed out the progress level
except:
    print("An error occured")



uploaded clinics.csv successfully
uploaded dropout_flags.csv successfully
uploaded feedback.csv successfully
uploaded interventions.csv successfully
uploaded patients.csv successfully
uploaded sessions.csv successfully


In [181]:
import os
print(os.listdir("../medoptix_data/raw"))


['.gitkeep', 'clinics.csv', 'dropout_flags.csv', 'feedback.csv', 'interventions.csv', 'patients.csv', 'sessions.csv']


In [182]:
print(os.listdir(folder_data))


['.gitkeep', 'clinics.csv', 'dropout_flags.csv', 'feedback.csv', 'interventions.csv', 'patients.csv', 'sessions.csv']


In [ ]:
import boto3
print(boto3.__version__)


1.39.3


- (B) - Download Data from AWS S3 → PostgreSQL

In [191]:
import pandas as pd
from sqlalchemy import create_engine
import boto3

# created a client
s3 = boto3.client("s3")

# defined our parameters
bucket = "flo-amdari-demo-etl"
prefix = "/medoptix/raw/"

# looped through our desired files and downloaded from s3
# Files to download
files = ["clinics.csv", "dropout_flags.csv", "feedback.csv", "interventions.csv", "patients.csv", "sessions.csv"]

# Download files
for file in files:
    s3.download_file(bucket, prefix + file, file)       # medoptix/raw/patients.csv
    print(f"Downloaded {file} successfully from S3")

Downloaded clinics.csv successfully from S3
Downloaded dropout_flags.csv successfully from S3
Downloaded feedback.csv successfully from S3
Downloaded interventions.csv successfully from S3
Downloaded patients.csv successfully from S3
Downloaded sessions.csv successfully from S3


In [169]:
response = s3.list_objects_v2(Bucket="flo-amdari-demo-etl")
for obj in response.get("Contents", []):
    print(obj["Key"])


./medoptix/etl/clinics.csv
./medoptix/etl/dropout_flags.csv
./medoptix/etl/feedback.csv
./medoptix/etl/interventions.csv
./medoptix/etl/patients.csv
./medoptix/etl/sessions.csv
./medoptix_data/processedclinics.csv
./medoptix_data/processeddropout_flags.csv
./medoptix_data/processedfeedback.csv
./medoptix_data/processedinterventions.csv
./medoptix_data/processedpatients.csv
./medoptix_data/processedsessions.csv
.medoptix_data/processedclinics.csv
.medoptix_data/processeddropout_flags.csv
.medoptix_data/processedfeedback.csv
.medoptix_data/processedinterventions.csv
.medoptix_data/processedpatients.csv
.medoptix_data/processedsessions.csv
/clinics.csv
/dropout_flags.csv
/feedback.csv
/interventions.csv
/med_optixclinics.csv
/med_optixdropout_flags.csv
/med_optixfeedback.csv
/med_optixinterventions.csv
/med_optixpatients.csv
/med_optixsessions.csv
/medoptix/etl/clinics.csv
/medoptix/etl/dropout_flags.csv
/medoptix/etl/feedback.csv
/medoptix/etl/interventions.csv
/medoptix/etl/patients.csv

In [194]:
response = s3.list_objects_v2(Bucket="flo-amdari-demo-etl")
for obj in response.get("Contents", []):
    print(obj["Key"])

/medoptix/raw/clinics.csv
/medoptix/raw/dropout_flags.csv
/medoptix/raw/feedback.csv
/medoptix/raw/interventions.csv
/medoptix/raw/patients.csv
/medoptix/raw/sessions.csv


### Data Modelling & Defining Foreign Key relationships
- patients (PK: patient_id, FK: clinic_id)
- sessions (PK: session_id, FK: patient_id)
- feedback (PK: feedback_id, FK: session_id)

TASK 1 - Model and Define the relationship for the remaining set of dataset

- clinics
- dropout_flags
- intervention.csv

TASK 2 - Create Schema for these dataset and upload them into postgres

ANSWER TO TASK 1 - Model and Define the relationship for the remaining datasets

- clinics (PK: clinic_id)
- dropout_flags (PK:patient_id, FK: patient_id)
- intervention.csv (Pk: intervention_id, FK: patient_id)

- (C) - Upload Data → PostgreSQL

In [222]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


# Load environment variables (recommended for security)
# Load environment variables from .env file
load_dotenv()

# Fetch variables
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")


# define a function with a postgres client
def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{user}:{password}@"
        f"{host}:{port}/{database}?"
    )
    return create_engine(db_url)

def upload_data():
    # Step 1: Read CSV files
    clinics = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\clinics.csv")
    patients = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\patients.csv")
    dropout_flags = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\dropout_flags.csv")
    interventions = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\interventions.csv")
    sessions = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\sessions.csv")
    feedback = pd.read_csv(r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\feedback.csv")
    
    
    

    # Step 2: Create database engine
    engine = get_db_engine()

    # Step 3: Upload in referential order with error handling
    with engine.begin() as connection:  # Automatically handles transactions
        # Chunk size for large datasets (adjust as needed)
        chunk_size = 1000

        # define an open connection for our DB client/engine
                
        # Upload clinics table
        clinics.to_sql(
            "clinics", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )
        
        # Upload patients table
        patients.to_sql(
            "patients", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'  # Faster for bulk inserts
        )

        # Upload dropout_flags table
        dropout_flags.to_sql(
            "dropout_flags", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

        # Upload interventions table
        interventions.to_sql(
            "interventions", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

        # Upload sessions table
        sessions.to_sql(
            "sessions", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )
        
        # Upload feedback table
        feedback.to_sql(
            "feedback", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )
        
        

        
    print("✅ Data uploaded successfully with relationships intact.")

if __name__ == "__main__":
    
    # Invote the function
    upload_data()

✅ Data uploaded successfully with relationships intact.


In [217]:
print(f"Connecting to DB as {user}@{host}:{port}/{database}")


Connecting to DB as avnadmin@pg-sql-host-for-flo-data-science-projects.d.aivencloud.com:14464/defaultdb


In [202]:
import os

file_path = r"C:\Florence Personal\Documents\Data Science Projects\Amdari Accelerate Internship Projects Folder\MedOptix_Project_Folder\notebooks\clinics.csv"
print(os.path.exists(file_path))  # Should return True


True


- (D) - Read Data → PostgreSQL - (Prepare data for EDA)

- EDA STEP 1: READING THE DATA FROM THE DATABASE for Exploratory Data Analysis

In [237]:
import os
import pandas as pd
from sqlalchemy import create_engine


# create function for postgres plugin (sqlalchemy)
def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{user}:{password}@"
        f"{host}:{port}/{database}?"
    )


    return create_engine(db_url)


# instantiate this function
engine = get_db_engine()


# Query to fetch data from the tables 
# defining the query for pandas
clinics_query = "SELECT * FROM clinics"
patients_query = "SELECT * FROM patients"
dropout_flags_query = "SELECT * FROM dropout_flags"
interventions_query = "SELECT * FROM interventions"
sessions_query = "SELECT * FROM sessions"
feedback_query = "SELECT * FROM feedback"

# Load data into pandas DataFrame --- convert that data into a dataframe
clinics_df = pd.read_sql(clinics_query, engine)
patients_df = pd.read_sql(patients_query, engine)
dropout_flags_df = pd.read_sql(dropout_flags_query, engine)
interventions_df = pd.read_sql(interventions_query, engine)
sessions_df = pd.read_sql(sessions_query, engine)
feedback_df = pd.read_sql(feedback_query, engine)

### D1- Display basic info and summary statistics for each table

In [ ]:
from IPython.display import display

# ================================================================
# ===================== 📊 Clinics Data ===========================
# ================================================================
print("📋 Clinics Data Overview (First 5 Rows)")
display(clinics_df.head())       

print("\n🧠 Clinics Data Info (Schema & Nulls)")
clinics_df.info()                  

print("\n📈 Clinics Data Statistics Summary")
display(clinics_df.describe())  

print("\n" + "="*70 + "\n")


# ================================================================
# ==================== 🧍‍♀️ Patients Data ===========================
# ================================================================
print("📋 Patients Data Overview (First 5 Rows)")
display(patients_df.head())       

print("\n🧠 Patients Data Info (Schema & Nulls)")
patients_df.info()                  

print("\n📈 Patients Data Statistics Summary")
display(patients_df.describe())  

print("\n" + "="*70 + "\n")


# ================================================================
# ================ 🚪 Dropout Flags Data ==========================
# ================================================================
print("📋 Dropout Flags Data Overview (First 5 Rows)")
display(dropout_flags_df.head())       

print("\n🧠 Dropout Flags Data Info (Schema & Nulls)")
dropout_flags_df.info()                  

print("\n📈 Dropout Flags Data Statistics Summary")
display(dropout_flags_df.describe())  

print("\n" + "="*70 + "\n")


# ================================================================
# ================ ✉️ Interventions Data ==========================
# ================================================================
print("📋 Interventions Data Overview (First 5 Rows)")
display(interventions_df.head())       

print("\n🧠 Interventions Data Info (Schema & Nulls)")
interventions_df.info()                  

print("\n📈 Interventions Data Statistics Summary")
display(interventions_df.describe())

print("\n" + "="*70 + "\n")


# ================================================================
# ===================== 📊 Sessions Data ===========================
# ================================================================
print("📋 Sessions Data Overview (First 5 Rows)")
display(sessions_df.head())       

print("\n🧠 Sessions Data Info (Schema & Nulls)")
sessions_df.info()                  

print("\n📈 Sessions Data Statistics Summary")
display(sessions_df.describe())  

print("\n" + "="*70 + "\n")


# ================================================================
# ==================== 🧍‍♀️ Feedback Data ===========================
# ================================================================
print("📋 Feedback Data Overview (First 5 Rows)")
display(feedback_df.head())       

print("\n🧠 Feedback Data Info (Schema & Nulls)")
feedback_df.info()                  

print("\n📈 Feedback Data Statistics Summary")
display(feedback_df.describe())  



📋 Clinics Data Overview (First 5 Rows)


,clinic_id,city,country,type,postcode,capacity,staff_count,speciality,avg_rating
0,1,London,UK,NHS,1184,109,26,Neuro-rehab,4.7
1,2,Manchester,UK,Private,39611,144,13,Neuro-rehab,3.7
2,3,Birmingham,UK,NHS,24119,153,30,Neuro-rehab,4.6
3,4,Berlin,DE,Private,42939,176,21,Sports PT,4.2
4,5,Munich,DE,NHS,12254,150,19,Sports PT,4.4



🧠 Clinics Data Info (Schema & Nulls)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   clinic_id    8 non-null      int64  
 1   city         8 non-null      object 
 2   country      8 non-null      object 
 3   type         8 non-null      object 
 4   postcode     8 non-null      int64  
 5   capacity     8 non-null      int64  
 6   staff_count  8 non-null      int64  
 7   speciality   8 non-null      object 
 8   avg_rating   8 non-null      float64
dtypes: float64(1), int64(4), object(4)
memory usage: 708.0+ bytes

📈 Clinics Data Statistics Summary


,clinic_id,postcode,capacity,staff_count,avg_rating
count,8.00000,8.000000,8.000000,8.000000,8.000000
mean,4.50000,35156.625000,153.250000,21.500000,4.237500
std,2.44949,27174.942744,24.574958,5.976143,0.373927
min,1.00000,1184.000000,109.000000,13.000000,3.700000
25%,2.75000,21152.750000,144.750000,18.250000,4.075000
50%,4.50000,32473.500000,151.500000,20.000000,4.300000
75%,6.25000,43460.250000,161.000000,26.500000,4.450000
max,8.00000,90786.000000,193.000000,30.000000,4.700000




📋 Patients Data Overview (First 5 Rows)


...

,patient_id,date,week,duration,pain_level,home_adherence_pc,satisfaction,therapist_id
count,70236.000000,70236,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000
mean,2498.827652,2025-01-20 12:53:29.515120640,6.490717,39.511134,1.982317,69.333775,6.490831,1099.411498
min,1.000000,2024-06-13 00:12:44,1.000000,20.000000,1.000000,0.000000,2.000000,1000.000000
25%,1247.000000,2024-10-20 00:12:44,3.000000,30.000000,1.000000,59.000000,6.000000,1049.000000
50%,2499.000000,2025-01-21 00:12:44,6.000000,40.000000,1.000000,69.000000,6.000000,1099.000000
75%,3751.000000,2025-04-21 00:12:44,9.000000,49.000000,3.000000,80.000000,7.000000,1149.000000
max,5000.000000,2025-09-02 00:12:44,12.000000,59.000000,8.000000,100.000000,10.000000,1199.000000
std,1444.925172,NaN,3.451124,11.516467,1.484108,14.846721,1.236411,57.610261




📋 Feedback Data Overview (First 5 Rows)


,feedback_id,session_id,comments,sentiment
0,435defde-e635-47a4-b33e-ca22aaa11e84,07b1b624-b7df-4be8-8d2b-e78952da2aa9,Would like longer sessions,0.361
1,cb6ef4c0-4924-4b99-8a91-cfe0f811d1b9,febf205b-d6e3-4cf6-bb17-88dcbd638a14,Therapist very helpful,0.475
2,a8e89e61-8e82-4b8a-87a6-db50e84ccfce,cf1ab3cb-36e5-4f4a-896c-5004a001c068,Feeling motivated,0.542
3,32c5cf20-2ffe-4388-83ca-8cf963a1b65f,d6b25084-03fa-4bec-811d-c4c8cc8c722e,Great progress today!,0.802
4,a17e9e47-1a3d-4c36-92b9-d2f8267bc4c7,e4003c56-d70a-448b-bcfa-ed7e34f50850,Noticeable improvement,0.459



🧠 Feedback Data Info (Schema & Nulls)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49165 entries, 0 to 49164
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   feedback_id  49165 non-null  object 
 1   session_id   49165 non-null  object 
 2   comments     49165 non-null  object 
 3   sentiment    49165 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.5+ MB

📈 Feedback Data Statistics Summary


,sentiment
count,49165.000000
mean,0.304360
std,0.356693
min,-0.440000
25%,0.000000
50%,0.459000
75%,0.542000
max,0.802000


#### STEPS - IN - EDA 

- step 1 - Data clening
    - step 1a - Identifying missing values in the data
    - step 1b - Handling missing values in the data

- step 2 - Data Transformation
- step 3 - Data Distribution
- step 4 - Conrrelation Analysis
- step 5 - Drop_out Analysis

### E. EDA → (Exploratory Data Analysis)

Steps in EDA

- Data Exploration
    - Head()
    - info()
    - describe()
- Data Cleaning
    - Identifying & Handling Missing Values
    - Identifying & Handling Duplicates

Data Transformation (Feature Engineering)
- Standardizing data types (text, date, replacing values)
- Log Transform (for highly skewed data e.g., income, prices)
- Binning (Convert continuous variables into categorical bins e.g., age ranges)
- Encoding (Convert categorical variables into numerical format (e.g., One-Hot, Label Encoding))

Data Distributions
- Histograms for distributions of numerical features (age, bmi etc)- 
- Also to check if the features are normally distributed or skewed (e.g., pain level or age should be somewhat evenly distributed just like we discovered in the last drop-in session).

- Boxplots to detect outliers & handle them.
- Pairplots to visualize relationships between multiple features.

- Correlation Analysis (dentify relationships between variables that may help in predictive modeling)
- Correlation Matrix to understand feature dependencies.

Heatmap for visual representation of correlations.
- Understanding Dropout Behavior
- investigate the correlation between features and patient dropout.

#### step 1a : Data Cleaning (Identifying missing values in the data) 


In [ ]:
#5. Check for Missing Values in All Tables
print("\nMissing Values in Clinics Data:")
print(clinics_df.isnull().sum())

print("\nMissing Values in Dropout_flags Data:")
print(dropout_flags_df.isnull().sum())

print("\nMissing Values in Feedback Data:")
print(feedback_df.isnull().sum())

print("\nMissing Values in Interventions Data:")
print(interventions_df.isnull().sum())

print("\nMissing Values in Patients Data:")
print(patients_df.isnull().sum())

print("\nMissing Values in Sessions Data:")
print(sessions_df.isnull().sum())



In [253]:
#5. Check for Missing Values in All Tables
print("\nMissing Values in Clinics Data:")
print(clinics_df.isnull().sum())

print("\nMissing Values in Dropout_flags Data:")
print(dropout_flags_df.isnull().sum())

print("\nMissing Values in Feedback Data:")
print(feedback_df.isnull().sum())

print("\nMissing Values in Interventions Data:")
print(interventions_df.isnull().sum())

print("\nMissing Values in Patients Data:")
print(patients_df.isnull().sum())

print("\nMissing Values in Sessions Data:")
print(sessions_df.isnull().sum())




Missing Values in Clinics Data:
clinic_id      0
city           0
country        0
type           0
postcode       0
capacity       0
staff_count    0
speciality     0
avg_rating     0
dtype: int64

Missing Values in Dropout_flags Data:
patient_id         0
dropout            0
dropout_week    4919
dtype: int64

Missing Values in Feedback Data:
feedback_id    0
session_id     0
comments       0
sentiment      0
dtype: int64

Missing Values in Interventions Data:
intervention_id    0
patient_id         0
sent_at            0
channel            0
message            0
responded          0
dtype: int64

Missing Values in Patients Data:
patient_id            0
age                   0
gender                0
bmi                   0
smoker                0
chronic_cond       3069
injury_type           0
signup_date           0
referral_source       0
consent               0
clinic_id             0
insurance_type        0
dtype: int64

Missing Values in Sessions Data:
session_id           0
p

-  After checking through the data, we can see that we have 4919 missing values under the dropout_week column in our dropout_flags_df and 3069 missing values under the chronic_cond column in the patient_df.

- To handle these missing values, we have to do a quick calculation of the percentage of the missing value.
- for chronic_cond under patients_df, the percentage of missing values is 61%,
- for dropout_week under dropout_flags_df, the percentage of missing values is 98%

- 61% of missing data in the patients dataset
- 98% of missing data in the dropout_flags dataset

- question: do we shed this data or we seek alternatives

- decision: 
    - step 1:
- does this column contain numerical data or categorical data?

- chronic_cond is a categorical data type (fill with mode)
- dropout_week is a numerical data type (fill with mean or median)


In [ ]:
# Unique values in the 'chronic_cond' column
print(patients_df['chronic_cond'].unique())

# Count of each value in 'chronic_cond'
print(patients_df['chronic_cond'].value_counts())

# Full list of values in 'chronic_cond'
print(patients_df['chronic_cond'].tolist())


[None 'Hypertension' 'Asthma' 'Cardio' 'Diabetes']
chronic_cond
Diabetes        741
Hypertension    552
Cardio          383
Asthma          255
Name: count, dtype: int64
[None, None, None, None, None, None, 'Hypertension', 'Hypertension', None, None, None, None, None, None, None, 'Asthma', None, 'Hypertension', None, None, 'Cardio', None, 'Diabetes', 'Asthma', None, None, None, None, None, None, 'Cardio', 'Diabetes', None, None, 'Diabetes', 'Cardio', None, None, 'Hypertension', 'Cardio', 'Cardio', None, None, None, None, 'Cardio', None, 'Diabetes', 'Cardio', 'Asthma', 'Diabetes', 'Cardio', None, None, None, 'Asthma', 'Cardio', None, None, None, None, 'Hypertension', None, 'Hypertension', None, None, None, 'Asthma', None, None, None, 'Cardio', 'Hypertension', 'Hypertension', None, 'Cardio', None, 'Diabetes', None, None, 'Asthma', 'Asthma', None, None, 'Cardio', 'Asthma', 'Hypertension', None, None, 'Cardio', 'Hypertension', 'Diabetes', 'Diabetes', None, None, 'Hypertension', None, 'Diab

In [ ]:
# Unique values in the 'dropout_week' column
print(dropout_flags_df['dropout_week'].unique())

# Count of each value in 'dropout_week'
print(dropout_flags_df['dropout_week'].value_counts())

# Full list of values in 'dropout_week'
print(dropout_flags_df['dropout_week'].tolist())


[nan 10.  9.  7.  8.]
dropout_week
10.0    50
9.0     22
8.0      7
7.0      2
Name: count, dtype: int64
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 10.0, nan, nan, nan, nan, nan, nan, nan, 10.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
# for chronic_cond column
mean_chronic_cond = patients_df['chronic_cond'].mean()
median_chronic_cond = patients_df['chronic_cond'].median()
mode_chronic_cond = patients_df['chronic_cond'].mode()

print(f"Mean Sentiment: {mean_chronic_cond}")
print(f"Median Sentiment: {median_chronic_cond}")
print(f"Mode Sentiment: {mode_chronic_cond.tolist()}")


#### step 1b : Data Cleaning  (Handling missing values in the data)

In [ ]:

# Example of filling missing values for categorical and numerical data
patients_df['chronic_cond'] = patients_df['chronic_cond'].fillna(patients_df['chronic_cond'].mode()[0])

In [ ]:

# Example of filling missing values for categorical and numerical data
dropout_flags_df['chronic_cond'] = dropout_flags_df['chronic_cond'].fillna(dropout_flags_df['chronic_cond'].mean()[0])

In [ ]:
# check if null values has been properly handled
clinics_df.isnull().sum()

In [ ]:
dropout_flags_df.isnull().sum()

In [ ]:
feedback_df.isnull().sum()

In [ ]:
interventions_df.isnull().sum()

In [ ]:
patients_df.isnull().sum()

In [ ]:
sessions_df.isnull().sum()

### E-3 - TASK 3 (Data Cleaning)
Identifying & Handling Duplicates

In [ ]:
clinics_df.duplicated().any()

In [ ]:
dropout_flags_df.duplicated().any()

In [ ]:
feedback_df.duplicated().any()

In [ ]:
interventions_df.duplicated().any()

In [ ]:
patients_df.duplicated().any()

In [ ]:
sessions_df.duplicated().any()

## F. Data transformation (Feature Engineering)
F-1 Standardizing data types (Signup_date)

In [ ]:
patients_df['signup_date'] = pd.to_datetime(patients_df['signup_date'])
sessions_df['date'] = pd.to_datetime(sessions_df['date'])

#### F-2 Binning (putting the age into categories)

In [ ]:
import numpy as np
# Add age group column for segmentation
bins = [0, 18, 35, 55, np.inf]
labels = ['0-18', '19-35', '36-55', '55+']
patients_df['age_group'] = pd.cut(patients_df['age'], bins=bins, labels=labels)
patients_df.head(2)

F-3 : TASK 4 (Data Transformation)

Log Transform (for highly skewed data e.g., income, prices)
Encoding (Convert categorical variables into numerical format (e.g., One-Hot, Label Encoding))

### G. Data Distribution
#### G-1 Histograms for distributions of numerical features

import seaborn as sns
import matplotlib.pyplot as plt

# Histogram for distribution of pain level
sns.histplot(sessions_df['pain_level'], kde=True)
plt.title('Distribution of Pain Level')
plt.show()

# Age distribution for patients (histogram)
sns.histplot(patients_df['age'], kde=True)
plt.title('Age Distribution')
plt.show()

### G-2 TASK 5 (Data Distribution)
Boxplots to detect outliers & handle them.
Pairplots to visualize relationships between multiple features.
H. CORRELATION ANALYSIS
H-1 TASK 6 (Correlation Analysis)
Correlation Analysis (dentify relationships between variables that may help in predictive modeling)
Correlation Matrix to understand feature dependencies.
Heatmap for visual representation of correlations.


In [ ]:
import numpy as np

In [120]:
A = np.array([[1, 2], [3, 4]])

B = np.array([[2, 1], [3, 1]])

In [121]:
M = A @ B

In [122]:
print(M)

[[ 8  3]
 [18  7]]


In [133]:
C = np.array([[2, 24], [2, 26], [2, 57], [1, 37]])

D = np.array([[1000], [150]])

In [134]:
E = C @ D

In [135]:
print(E)

[[ 5600]
 [ 5900]
 [10550]
 [ 6550]]


In [137]:
bill = ([[3913], [2682], [8617], [64400]])

In [140]:
error = bill - E

In [141]:
error

array([[-1687],
       [-3218],
       [-1933],
       [57850]])

In [ ]:
billpred = constant([[2000, 3600], [2000, 3900], [2000, 8550], [1000, 5550]])